Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [GitHub](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [ri.gov](http://www.governor.ri.gov/newsroom/speeches/index.php).
<hr>

## State of the State Word Scrape

In [88]:
import glob
import pandas as pd
import numpy as np

from nltk import word_tokenize, pos_tag, pos_tag_sents

In [55]:
files = glob.glob('../input/*.txt')

# getting all words in each speech

for file in files:

    speech = open(file)
    year = file[9:13]
    wds = {}
    
    for line in speech:
        
        line = line.replace('.','')
        line = line.replace(',','')
        line = line.replace('"','')
        line = line.replace(';','')
        line = line.replace('“','')
        line = line.replace(':','')
        line = line.replace('(','')
        line = line.replace(')','')
        line = line.replace('0','')
        line = line.replace('1','')
        line = line.replace('2','')
        line = line.replace('3','')
        line = line.replace('4','')
        line = line.replace('5','')
        line = line.replace('6','')
        line = line.replace('7','')
        line = line.replace('8','')
        line = line.replace('9','')
        line = line.replace('–','')
        line = line.replace('#','')
        line = line.replace('”','')
        line = line.replace('-','')
        line = line.replace('$','')
        line = line.replace("’","'")
        line = line.replace("s'",'')
        line = line.replace("'s",'')
        line = line.rstrip().lower()
        words = line.split()
        
        for w in words:
            
            wds[w] = wds.get(w,0) + 1

    # Creating a csv

    df = pd.DataFrame(list(wds.items()))
    
    df.columns = ['words', 'count']
    df['POSTags'] = pos_tag_sents(df['words'].apply(word_tokenize).tolist())

    df.to_csv('../output/word_count/' + year + '_word_count.csv')
